# Módulo 4 - Jupyter Avançado e 
#   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Reprodutibilidade

Esta parte do minicurso tem o objetivo de aprofundar em características avançadas do Jupyter

- IPython como superconjunto do Python
- Mágicas, Exibições e Extensões
- Widgets interativos

Em seguida:

- Ciência Aberta com Reprodutibilidade

## Aula 1 - IPython

Este notebook em si tem o objetivo de apresentar o IPython como um superconjunto do Python. Ou seja, ele estende a sintaxe do Python para adicionar novas funcionalidades.

### IPython

A execução das células de um notebook ocorre em um **kernel**.

No nosso caso, por usarmos um notebook Python, o kernel que está executando o código é o IPython.

O IPython inicialmente foi criado como um programa REPL e evoluiu até se tornar o Jupyter.

Atualmente o Jupyter suporta diversos outros kernels para outras linguagens.

É possível obter o objeto que representa o kernel em execução através do seguinte comando:

In [1]:
get_ipython()

A execução de células no IPython oferece diversas extensões. Aqui serão apresentadas as seguintes:

- Extensão 1: Variáveis Especiais
- Extensão 2: Expressões Bang
- Extensão 3: Mágicas de linha
- Extensão 4: Mágicas de célula
- Extensão 5: Consulta à documentação


### Extensão 1: Variáveis especiais

Ao executar uma célula, o IPython cria novas variáves e armazena valores que podem ser referenciados posteriormente.

Com `_` é possível acessar o resultado da célula anterior:

In [2]:
a = 5
a

5

In [3]:
_ + 1

6

Se quiser acessar o resultado de alguma célula específica (`x`), é possível usar o dicionário `Out[x]` ou mesmo a variável `_x`:

In [4]:
Out[2], _3

(5, 6)

O dicionário `In` permite obter o código da célula:

In [5]:
In[3]

'_ + 1'

### Extensão 2: Expressão Bang

Executa comandos no sistema e retorna a saída do comando como resultado.

Já foi apresentada anteriormente como:
```
!pip install fuzzywuzzy
```

Mas podemos usar qualquer comando de sistema:

In [6]:
!ls ../dataset/ 

spotify_artists_info_complete.tsv  spotify_charts_complete.tsv
spotify_artists_info_edited.csv    spotify_hits_dataset_complete.tsv


É o equivalente a

```python
get_ipython().system('ls ../dataset/')
```

A seguinte célula atribui a lista de arquivos a variável files.

Em seguida, ela percorre os arquivos, pegando nome e número de linhas.

In [7]:
files = !ls ../dataset/
for name in files:
    content = open("../dataset/" + name).readlines()
    print(name, '--', len(content))

spotify_artists_info_complete.tsv -- 626
spotify_artists_info_edited.csv -- 626
spotify_charts_complete.tsv -- 10401
spotify_hits_dataset_complete.tsv -- 1285


Como está em uma atribuição, a expressão bang é transformada em um uma função ligeiramente diferente:

```python
files = get_ipython().getoutput('ls ../dataset/')
```

Comparando:

- `get_ipython().system('ls')` imprime resultado do comando
- `get_ipython().getoutput('ls')` retorna resultado do comando

### Extensão 3: Mágica de linha

Tem o objetivo de modificar a forma de executar uma linha.

In [8]:
%history -n

   1: get_ipython()
   2:
a = 5
a
   3: _ + 1
   4: Out[2], _3
   5: In[3]
   6: !ls ../dataset/
   7:
files = !ls ../dataset/
for name in files:
    content = open("../dataset/" + name).readlines()
    print(name, '--', len(content))
   8: %history -n


Essa mágica é transformada em:
```python
get_ipython().run_line_magic('history', '-n')
```


Essencialmente, mágicas de linha são funções chamadas de uma forma um pouco diferente.

#### Mágicas de linha mais usadas

| Mágica | Notebooks | Descrição | 
|:------ |:---------:|:-------- |
| `%matplotlib` | 156.353 | Configura matplotlib para funcionar interativamente |
| `%load_ext` | 14.216 | Carrega uma extensão IPython pelo nome do módulo |
| `%autoreload` | 12.920 | Recarrega módulos automaticamente |
| `%pylab` | 8.183 | Carrega numpy e matplotlib |
| `%time` | 5.172 | Mede a duração da execução de uma expressão | 
| `%config` | 4.805 | Configura o IPython |
| `%pinfo` | 3.253 | Acessa a documentação de um objeto | 
| `%run` | 2.474 | Executa um arquivo Python dentro do IPython |
| `%timeit` | 2.318 | Tira a mediana de várias execuções de `%time` | 
| `%reload_ext` | 1.635 | Recarrega uma extensão IPython|

Extraídas de um conjunto de 173.256 notebooks que usavam mágicas de linha em um estudo que coletou 1.450.071 notebooks do GitHub [Pimentel et al. 2021].

### Extensão 4: Mágica de célula

Tem o objetivo de modificar a forma de executar uma célula.

In [9]:
%%html
<a href="../dataset/spotify_artists_info_complete.tsv">artists</a>
<a href="../dataset/spotify_charts_complete.tsv">charts</a>
<a href="../dataset/spotify_hits_dataset_complete.tsv">hits</a>

Essa mágica é transformada em:

```python
get_ipython().run_cell_magic('html', '', '<a href...</a>\n')
```

Considerações:
- Mágica de célula deve ser usada no topo da célula
- Pode aceitar parâmetros semelhantes ao da mágica de linha
- Permite combinar diversas linguagens em um notebook

#### Desvantagem de mágicas de célula

Como elas modificam a forma de executar a célula inteira, muitas vezes não é possível combinar a geração do conteúdo com Python para gerar resultados programando.

Por exemplo, se quisermos gerar o HTML com links a partir de arquivos na pasta:

In [10]:
html = '<br>'.join(
    f'<a href="../dataset/{name}">{name}</a>'
    for name in files
)
html

'<a href="../dataset/spotify_artists_info_complete.tsv">spotify_artists_info_complete.tsv</a><br><a href="../dataset/spotify_artists_info_edited.csv">spotify_artists_info_edited.csv</a><br><a href="../dataset/spotify_charts_complete.tsv">spotify_charts_complete.tsv</a><br><a href="../dataset/spotify_hits_dataset_complete.tsv">spotify_hits_dataset_complete.tsv</a>'

In [11]:
%%html
{html}

Usando o comando direto no kernel, é possível chegar ao resultado:

In [12]:
get_ipython().run_cell_magic('html', '', html)

Ou, mais diretamente, usando funções de exibição do IPython (`display` e `HTML`):

In [13]:
from IPython.display import HTML
display(HTML(html))

#### Mágicas de célula mais usadas

| Mágica | Notebooks | Descrição | 
|:------ |:---------:|:-------- |
| `%%time` | 13.129 | O mesmo que `%time`, mas para a célula | 
| `%%html` | 2.944 | Exibe célula como HTML | 
| `%%writefile` | 2.937 | Escreve o conteúdo de uma célula em um arquivo | 
| `%%bash` | 2.850 | Executa a célula com bash em um subprocesso |
| `%%timeit` | 2.628 | O mesmo que `%timeit`, mas para a célula |
| `%%javascript` | 2.447 | Executa a célula em Javascript no navegador | 
| `%%sql` | 1.822 | Realiza consulta SQL |
| `%%R` | 1.285 | Executa a célula com R em um subprocesso | 
| `%%capture` | 1.131 | Captura stdout, stderr e IPython display da execução |
| `%%cython` | 1.117 | Compila e importa todas as funções definidas em cython |

Extraídas de um conjunto de 33.541 notebooks que usavam mágicas de célula em um estudo que coletou 1.450.071 notebooks do GitHub [Pimentel et al. 2021].

### Mágicas disponíveis

Nem todas essas mágicas estão disponíveis em uma instalação inicial do Jupyter. Muitas vêm de bibliotecas externas. Para saber quais estão disponíveis, é possível usar uma mágica de linha.

In [14]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

### Extensão 5: Consulta à documentação

In [15]:
len?

Signature: len(obj, /)
Docstring: Return the number of items in a container.
Type:      builtin_function_or_method


Ao adicionar uma interrogação em um nome e executar a célula, o IPython exibe a documentação. 

Essa consulta é convertida em:

```python
get_ipython().run_line_magic('pinfo', 'len')
```

Que é o mesmo que:
```python
%pinfo len
```

A consulta pode ser feita em qualquer nome ou atributo:
- Funções
- Classes
- Variáveis
- Mágicas

Por exemplo:

In [16]:
%lsmagic?

Docstring: List currently available magic functions.
File:      ~/anaconda3/lib/python3.8/site-packages/IPython/core/magics/basic.py


Além de consultar a documentação, é possível consultar o código fonte de qualquer função, classe ou mágica definida em Python, através do uso de duas interrogações.

In [17]:
%lsmagic??

Source:
    @line_magic
    def lsmagic(self, parameter_s=''):
        """List currently available magic functions."""
        return MagicsDisplay(self.shell.magics_manager, ignore=[])
File:   ~/anaconda3/lib/python3.8/site-packages/IPython/core/magics/basic.py


Essa consulta é convertida em:

```python
get_ipython().run_line_magic('pinfo2', 'len')
```

Que é o mesmo que:
```python
%pinfo2 len
```

### Transformar IPython em Python

Todas as operações do IPython são convertidas em Python para execução. Para saber como ocorre cada transformação, é possível executar o método `transform_cell` do kernel.

In [18]:
get_ipython().transform_cell('len??')

"get_ipython().run_line_magic('pinfo2', 'len')\n"

## Conclusão

Este notebook apresentou variáveis especiais, bang expressions (!), mágicas de linha (%), mágicas de célula (%%), e consultas a documentação (?).

O próximo notebook ([4.2.Magica.ipynb](4.2.Magica.ipynb)) apresenta a definição de mágicas e extensão para o IPython.

E o seguinte ([4.3.Exibicao.ipynb](4.3.Exibicao.ipynb)) utiliza a extensão para apresentar como estender a visualização no IPython e criar visualizações ricas.

Na apresentação do dia 04/10, vamos pular esses dois notebooks por questão de tempo, e iremos direto para o último notebook desta parte ([4.4.Widget.ipynb](4.4.Widget.ipynb)), que apresenta como criar um widget interativo simples.

Mas recomendamos o acesso do conteúdo deles por meio do [Capítulo](https://sol.sbc.org.br/livros/index.php/sbc/catalog/view/67/292/544-1) e [Repositório](https://github.com/opgabriel/jai2021-jupyter) do JAI. O repositório do JAI também possui um notebook extra que ensina a fazer Widgets personalizados.
